## 작동 수도 코드

서버 작동 방식
- t 분마다 한 번씩 현 시점에서 n초 이후의 온도를 예측하고 결과(pred_temp)를 받아옴
- 알고리즘의 판단에 따라 작동 변수를 조정한 뒤 조정된 정보를 서버에 넘겨 값을 업데이트 함

사용 변수
- 현재 온도: curr_temp
- 예측 온도: pred_temp
- 현재 환기온도(pre-defined): max_thres
- 현재 난방온도(pre-defined): min_thres
- 예측 시점 - 현재 시점: n


작동 변수(상호 반대)
- 음의 방향 작용(온도를 내리는)
  * 차광스크린, 차열스크린, 에너지스크린(수평, 수직), 배기팬, 천창, (유동팬) 값 증가
- 양의 방향으로 작용(온도를 올리는)
  * 팬코일 값 증가

***

### 1. 경계: 최소 온도와 최대 온도

변수 설명
- 실제 서버에서 주기적으로 가져옴
- 작동 결과를 서버에 업데이트 함

In [2]:
# 현재 온도는 25도, n초 후 예측된 온도는 35도, 현 시점에서 최대 온도(환기온도)는 30도, 최소 온도(난방온도)는 18도
curr_temp = 25
pred_temp = 35
max_thres = 30
min_thres = 18

고민 거리
- 이거 넣을지 말지 결정
  * RRS = 0.7 # 변동에 너무 심하게 변하지 않게 쿠션감을 주는 값이라고 함! ex) 10만큼 변화해야하면 7만큼만 변화를 주는 것
- 스크린이 0에서 100으로 바로 증가하는 게 아니라 증가 term이 있을 텐데, 이걸 어떻게 처리해야할지
  * 데이터 보니까 대부분 바로 했던데 우리도 그냥 해도 될듯

In [11]:
curr_temps = [24, 26, 25, 23]
pred_temps = [30, 31, 24, 18]
max_thres = [29, 32, 29, 28]
min_thres = [20, 22, 19, 19]

RRS = 0.7

minus_affect = """
배기팬, 유동팬, 천창: 1(작동)
팬코일: 0(정지)
차광스크린: 100%(증가)
차열스크린: 100%(증가)
에너지스크린(수평): 100%(증가)
에너지스크린(수직): 100%(증가)

"""+"="*30

plus_effect = """
배기팬, 유동팬, 천창: 0(정지)
팬코일: 1(작동)
차광스크린: 0%(감소)
차열스크린: 0%(감소)
에너지스크린(수평): 0%(감소)
에너지스크린(수직): 0%(감소)

"""+"="*30

In [13]:
# main

# Input code needed
# while True:
    # workload_next=input()
    # if workload_next=='q': break

for pred_temp in pred_temps:
    print("="*30)
    
    # 동일 시점
    idx=pred_temps.index(pred_temp)
    
    # 온도 차
    temp_diff = pred_temp - curr_temps[idx]
    
    # RRS 적용 시 
    ## temp_diff*=RRS
    
    ## 이거 작동 틀려서 다시 해야함(현재 온도가 중요한 게 아니라 예측온도와 최대, 최소 온도가 중요한 것!)
    # 현재 온도 > 예측 온도 > 최소 온도: 하강
    if (curr_temps[idx] > pred_temp) and (temp_diff < 0):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx]))
        print("작동: 음의 방향")
        print("------------------------")
        print(minus_affect)
        continue
    
    # 현재 온도, 최소 온도 > 예측 온도 : 하강(최소가 목표)
    elif (temp_diff < 0 ) and (min_thres[idx] > pred_temp):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx]))
        print("작동: 음의 방향 - 최소가 목표")
        print("------------------------")
        print(minus_affect)
        continue
    
    # 최대 온도 > 예측 온도 > 현재 온도: 무족권 상승
    elif (max_thres[idx] > pred_temp) and (temp_diff > 0):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx]))
        print("작동: 양의 방향")
        print("------------------------")
        print(plus_effect)
        continue
    
    # 예측 온도 > 최대 온도 , 현재 온도: 상승(최대가 목표))
    elif (pred_temp > max_thres[idx]) and (temp_diff > 0):
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx]))
        print("작동: 양의 방향 - 최대가 목표")
        print("------------------------")
        print(plus_effect)
        continue
    
    else: 
        print("현재 온도: %d\n예측 온도: %d\n최대 온도: %d\n최소 온도: %d"%(curr_temps[idx], pred_temp, max_thres[idx], min_thres[idx]))
        print("에러에러")

    

현재 온도: 24
예측 온도: 30
최대 온도: 29
최소 온도: 20
작동: 양의 방향 - 최대가 목표
------------------------

배기팬, 유동팬, 천창: 0(정지)
팬코일: 1(작동)
차광스크린: 0%(감소)
차열스크린: 0%(감소)
에너지스크린(수평): 0%(감소)
에너지스크린(수직): 0%(감소)

현재 온도: 26
예측 온도: 31
최대 온도: 32
최소 온도: 22
작동: 양의 방향
------------------------

배기팬, 유동팬, 천창: 0(정지)
팬코일: 1(작동)
차광스크린: 0%(감소)
차열스크린: 0%(감소)
에너지스크린(수평): 0%(감소)
에너지스크린(수직): 0%(감소)

현재 온도: 25
예측 온도: 24
최대 온도: 29
최소 온도: 19
작동: 음의 방향
------------------------

배기팬, 유동팬, 천창: 1(작동)
팬코일: 0(정지)
차광스크린: 100%(증가)
차열스크린: 100%(증가)
에너지스크린(수평): 100%(증가)
에너지스크린(수직): 100%(증가)

현재 온도: 23
예측 온도: 18
최대 온도: 28
최소 온도: 19
작동: 음의 방향 - 최소가 목표
------------------------

배기팬, 유동팬, 천창: 1(작동)
팬코일: 0(정지)
차광스크린: 100%(증가)
차열스크린: 100%(증가)
에너지스크린(수평): 100%(증가)
에너지스크린(수직): 100%(증가)

